In [ ]:
# Download da biblioteca auxiliar

!wget https://github.com/ufrpe-2021-2-IA/music-dataset/releases/download/v1.1.2/mgd-1.1.2-py3-none-any.whl
%pip install mgd --find-links /content/

# Testes: Validação Cruzada

Como já possuímos a divisão dos folds, precisamos implementar o nosso próprio loop de CV. Além disso, também vamos otimizar os hiperparâmetros para cada um dos algoritmos.

In [ ]:
#@title Configurações do Experimento
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from mgd import utils

import pathlib

import numpy as np
from sklearn.base import is_classifier, clone
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# raw: sem normalização
# min_max: com normalização min-max
# standardized: com normalização standardization (média e variância)
dataset = 'min_max' #@param ["raw", "min_max", "standardized"]
experiment = 'experiment-1' #@param ["experiment-1", "experiment-2", "experiment-3", "experiment-4", "experiment-5"]
selected_features = "75Features" #@param ["all", "100Features", "75Features"]
utils.download_experiments(dataset)

## Seleção Características Mais Importantes

In [ ]:
if selected_features == 'all':
    selected_features = None
elif selected_features == '75Features':
    selected_features = ['spectral_roll_of_skew',
                         'mfcc_10_max',
                         'tonnetz_1_min',
                         'mfcc_4_mean',
                         'mfcc_4_median',
                         'mfcc_9_std',
                         'tonnetz_1_max',
                         'mfcc_2_median',
                         'tonnetz_2_max',
                         'mfcc_12_std',
                         'mfcc_2_mean',
                         'mfcc_1_max',
                         'mfcc_6_median',
                         'mfcc_7_std',
                         'mfcc_6_mean',
                         'spectral_centroid_min',
                         'mfcc_9_mean',
                         'mfcc_11_min',
                         'spectral_roll_of_min',
                         'mfcc_9_median',
                         'mfcc_12_max',
                         'tonnetz_3_min',
                         'tonnetz_4_min',
                         'tonnetz_4_max',
                         'mfcc_7_max',
                         'spectral_flatness_mean',
                         'mfcc_12_mean',
                         'mfcc_12_median',
                         'mfcc_11_mean',
                         'mfcc_8_median',
                         'tonnetz_3_max',
                         'mfcc_11_median',
                         'mfcc_8_mean',
                         'mfcc_6_std',
                         'tonnetz_6_max',
                         'tonnetz_5_max',
                         'tonnetz_5_min',
                         'mfcc_4_std',
                         'mfcc_4_max',
                         'spectral_flatness_max',
                         'mfcc_7_min',
                         'mfcc_5_std',
                         'tonnetz_6_min',
                         'mfcc_5_min',
                         'mfcc_8_max',
                         'mfcc_3_std',
                         'mfcc_6_max',
                         'mfcc_5_median',
                         'mfcc_7_mean',
                         'mfcc_7_median',
                         'mfcc_5_mean',
                         'tonnetz_2_std',
                         'spectral_flatness_std',
                         'mfcc_2_max',
                         'tonnetz_1_std',
                         'spectral_roll_of_std',
                         'mfcc_3_min',
                         'tonnetz_3_std',
                         'mfcc_1_median',
                         'mfcc_1_mean',
                         'mfcc_3_max',
                         'mfcc_5_max',
                         'tonnetz_4_std',
                         'mfcc_3_median',
                         'tonnetz_6_std',
                         'mfcc_1_min',
                         'mfcc_3_mean',
                         'tonnetz_5_std',
                         'spectral_centroid_median',
                         'spectral_centroid_mean',
                         'spectral_roll_of_median',
                         'spectral_roll_of_mean',
                         'spectral_centroid_std',
                         'spectral_roll_of_max',
                         'spectral_centroid_max']
elif selected_features == '100Features':
    selected_features = ['mfcc_5_skew',
                         'tonnetz_1_kurtosis',
                         'tonnetz_3_kurtosis',
                         'mfcc_4_kurtosis',
                         'mfcc_11_skew',
                         'tonnetz_5_kurtosis',
                         'spectral_flatness_min',
                         'mfcc_10_skew',
                         'mfcc_7_skew',
                         'mfcc_6_skew',
                         'tonnetz_2_min',
                         'mfcc_9_max',
                         'spectral_flatness_skew',
                         'spectral_flatness_median',
                         'mfcc_8_min',
                         'mfcc_4_min',
                         'mfcc_1_std',
                         'mfcc_6_min',
                         'mfcc_8_std',
                         'mfcc_10_mean',
                         'mfcc_10_median',
                         'mfcc_2_std',
                         'mfcc_9_min',
                         'mfcc_11_std',
                         'mfcc_10_std',
                         'spectral_roll_of_skew',
                         'mfcc_10_max',
                         'tonnetz_1_min',
                         'mfcc_4_mean',
                         'mfcc_4_median',
                         'mfcc_9_std',
                         'tonnetz_1_max',
                         'mfcc_2_median',
                         'tonnetz_2_max',
                         'mfcc_12_std',
                         'mfcc_2_mean',
                         'mfcc_1_max',
                         'mfcc_6_median',
                         'mfcc_7_std',
                         'mfcc_6_mean',
                         'spectral_centroid_min',
                         'mfcc_9_mean',
                         'mfcc_11_min',
                         'spectral_roll_of_min',
                         'mfcc_9_median',
                         'mfcc_12_max',
                         'tonnetz_3_min',
                         'tonnetz_4_min',
                         'tonnetz_4_max',
                         'mfcc_7_max',
                         'spectral_flatness_mean',
                         'mfcc_12_mean',
                         'mfcc_12_median',
                         'mfcc_11_mean',
                         'mfcc_8_median',
                         'tonnetz_3_max',
                         'mfcc_11_median',
                         'mfcc_8_mean',
                         'mfcc_6_std',
                         'tonnetz_6_max',
                         'tonnetz_5_max',
                         'tonnetz_5_min',
                         'mfcc_4_std',
                         'mfcc_4_max',
                         'spectral_flatness_max',
                         'mfcc_7_min',
                         'mfcc_5_std',
                         'tonnetz_6_min',
                         'mfcc_5_min',
                         'mfcc_8_max',
                         'mfcc_3_std',
                         'mfcc_6_max',
                         'mfcc_5_median',
                         'mfcc_7_mean',
                         'mfcc_7_median',
                         'mfcc_5_mean',
                         'tonnetz_2_std',
                         'spectral_flatness_std',
                         'mfcc_2_max',
                         'tonnetz_1_std',
                         'spectral_roll_of_std',
                         'mfcc_3_min',
                         'tonnetz_3_std',
                         'mfcc_1_median',
                         'mfcc_1_mean',
                         'mfcc_3_max',
                         'mfcc_5_max',
                         'tonnetz_4_std',
                         'mfcc_3_median',
                         'tonnetz_6_std',
                         'mfcc_1_min',
                         'mfcc_3_mean',
                         'tonnetz_5_std',
                         'spectral_centroid_median',
                         'spectral_centroid_mean',
                         'spectral_roll_of_median',
                         'spectral_roll_of_mean',
                         'spectral_centroid_std',
                         'spectral_roll_of_max',
                         'spectral_centroid_max']


## 10-Fold Cross-Validation

In [ ]:
@ignore_warnings(category=ConvergenceWarning)
def cv_train(clf, 
             path_to_experiment,
             param_grid):
  p = pathlib.Path(path_to_experiment).absolute()
  folds = [utils.prepare_sklearn(str(p.joinpath(f'fold-{i}.csv')), 
                                 selected_features) for i in range(1, 11)]

  reports = []
  best_models = []

  def _prepare_train_validation(fold_index):
    folds_ = folds[:fold_index] + folds[fold_index + 1:]

    # Conjunto de validação
    X_val, Y_val = folds[fold_index]

    # Conjunto de treino
    X_train, Y_train = folds_[0]

    for X, Y in folds_[1:]:
      X_train = np.concatenate((X_train, X))
      Y_train = np.concatenate((Y_train, Y))

    return X_train, X_val, Y_train, Y_val

  # Loop de CV
  for i in range(0, 10):
    print(f'Started Fold {i+1}')
    # Preparar os conjunto de treino e validação
    X_train, X_val, Y_train, Y_val = _prepare_train_validation(i)

    # Criação do GridSearch para otimizar os parâmetros nessa iteração
    # Nested-Cross Validation com K=3
    estimator = clone(clf)
    clf_ = GridSearchCV(estimator=estimator, 
                        param_grid=param_grid,
                        cv=3,
                        refit=True)
    
    # Buscar pelos parâmetros
    clf_.fit(X_train, Y_train)

    # Guardar melhor modelo
    best_models.append(clf_.best_estimator_)

    # Calcular métricas no conjunto de validação
    predictions = best_models[-1].predict(X_val)
    report = classification_report(Y_val, 
                                   predictions, 
                                   digits=3, 
                                   output_dict=True)
    reports.append(report)

    print(f'Finished Fold {i+1}')
    print(reports[-1]['weighted avg'])
    print('-'*10)

  return best_models, reports


In [ ]:
def to_dataframe(reports):
  data = {
      'precision': [],
      'recall': [],
      'f1-score': []
  }

  for r in reports:
    r_ = r['weighted avg']

    for k in list(data.keys()):
      data[k].append(r_[k])

  return pd.DataFrame(data)
  

## ANN

In [ ]:
models, reports = cv_train(clf=MLPClassifier(),
                           path_to_experiment=f'/content/{dataset}/{experiment}',
                           param_grid={'hidden_layer_sizes': [(149, 40), (100, 40)],
                                       'activation': ['relu'],
                                       'solver': ['adam'],
                                       'alpha': [1e-4, 1e-5],
                                       'max_iter': [200],
                                       'learning_rate_init': [1e-3, 1e-4],
                                       'n_iter_no_change': [500]})
df = to_dataframe(reports)

In [ ]:
# Média das métricas para o experimento
df.mean()

In [ ]:
# Desvio padrão das métricas para o experimento
df.std()

## SVM

In [ ]:
models, reports = cv_train(clf=svm.SVC(),
                           path_to_experiment=f'/content/{dataset}/{experiment}',
                           param_grid={'C': [0.25, 0.5, 1.0],
                                       'decision_function_shape': ['ovo', 'ovr'],
                                       'kernel': ['rbf', 'linear'],
                                       'max_iter': [200]})
df = to_dataframe(reports)

In [ ]:
df.mean()

In [ ]:
df.std()

## K-NN

In [ ]:
models, reports = cv_train(KNeighborsClassifier(),
                      f'/content/{dataset}/{experiment}',
                      {'n_neighbors': [3, 5, 8, 10, 15],
                       'leaf_size': [20, 30, 40],
                       'algorithm': ['auto', 'ball_tree', 'kd_tree'],
                       'metric': ['minkowski', 'l1', 'l2', 'manhattan']})
df = to_dataframe(reports)

In [ ]:
df.mean()

In [ ]:
df.std()

## Árvores de Decisão

In [ ]:
models, reports = cv_train(tree.ExtraTreeClassifier(),
                      f'/content/{dataset}/{experiment}',
                      {'criterion': ['gini', 'entropy'],
                       'splitter': ['best', 'random'],
                       'min_samples_split': [2, 4, 8, 10],
                       'min_samples_leaf': [1, 2, 4],
                       'ccp_alpha': [0.0, 0.25, 0.5],
                       'max_features': ['auto', 'sqrt', 'log2']})
df = to_dataframe(reports)

In [ ]:
df.mean()

In [ ]:
df.std()